In [119]:
import ast

import numpy as np
import pandas as pd
import pysgpp as pysgpp
from sklearn.model_selection import train_test_split


from pysgpp.extensions.datadriven.learner import Types
from pysgpp.extensions.datadriven.learner import LearnerBuilder
from pysgpp.extensions.datadriven.uq.plot.plot1d import plotSG1d
from pysgpp.extensions.datadriven.uq.plot.plot2d import plotSG2d,plotGrid2d
from pysgpp.extensions.datadriven.uq.plot.plot3d import plotSG3d
from pysgpp.extensions.datadriven.uq.plot.plotGrid import plotGrid

from pysgpp.extensions.datadriven.learner.folding import RandomFoldingPolicy

#from learner.builder.RegressorSpecificationDescriptor.FoldingStrategy import RANDOM

import matplotlib.pyplot as plt
from sklearn import preprocessing
plt.style.use('ggplot')
%pylab inline
pylab.rcParams['figure.figsize'] = (20, 12)

Populating the interactive namespace from numpy and matplotlib


In [59]:
#filename = 'output_100_configs.csv'
#filename = '2D_param_space_times.csv'
filename = '1000_runs_8D_param_room.csv'


df = pd.read_csv(filename, header = None, names = ['params', 'time'])
df.params = df.params.apply(ast.literal_eval)

for param in df.iloc[0].params.keys():
    df[param] = df.params.apply(lambda row: row[param])
    
df['density'] = (df.MOL_X*df.MOL_Y)/(df.DOMAIN_SIZE_X*df.DOMAIN_SIZE_Y)
del df['params']
df = df[df.time > 0]

In [60]:
def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

In [61]:
columns = ['DOMAIN_SIZE_X', 'DOMAIN_SIZE_Y', 'MOL_X', 'MOL_Y', 'LINKED_CELL_SIZE_X', 'LINKED_CELL_SIZE_Y',
          'BLOCK_SIZE', 'CUTOFF_RADIUS', 'density','time']

In [62]:
params_ = ['CUTOFF_RADIUS', 'density', 'MOL_Y','MOL_X','DOMAIN_SIZE_Y'
          ,'DOMAIN_SIZE_X','MOL_X','LINKED_CELL_SIZE_X', 'LINKED_CELL_SIZE_Y']

In [63]:
params_2 = ['density','LINKED_CELL_SIZE_Y']

In [64]:
params_4 = ['density','CUTOFF_RADIUS']

In [79]:
#df[df.time > 0][params_]

In [86]:
normalized_df = normalize(df[df.time > 0][columns])
train, test = train_test_split(normalized_df, test_size=0.2)
num_samples = train.shape[0]

In [87]:
X_train = train[params_].values
Y_train = train['time'].values

In [88]:
X_test = test[params_].values
Y_test = test['time'].values

In [93]:
X_train.shape

(599, 9)

In [154]:
for levels in range(1, 3):
    builder = LearnerBuilder()
    builder.buildRegressor()
    builder.withTrainingDataFromNumPyArray(X_train, Y_train)
    builder.withTestingDataFromNumPyArray(X_test, Y_test)
    builder = builder.withGrid().withBorder(Types.BorderTypes.NONE)
    builder.withLevel(levels)
    builder = builder.withSpecification()
    builder.withLambda(0.000001).withAdaptPoints(100)
    
    #builder.withRandomFoldingPolicy().withLevel(4)

    builder.withIdentityOperator()
    builder = builder.withStopPolicy()
    builder = builder.withCGSolver()
    builder.withAccuracy(0.0001)
    builder.withImax(1000)
    learner = builder.andGetResult()
    
    

    #learner.learnData()
    #learner.learnDataWithFolding()
    learner.learnDataWithTest()
    print 'Test accuracy:' + str(learner.testAccuracy)
    print 'Level: %i' % levels
    print 'L2Norm: %f' % learner.getL2NormError()
    print 'MaxError: %f' % learner.getMaxError()
    print 'MinError: %f' % learner.getMinError()
    print 'Error: %f' % learner.evalError(learner.dataContainer,learner.alpha)
    print 'RMSerror: %f' % learner.error.RMSNorm()
    print 'Grid size BEFORE refinement: %i' % learner.grid.getSize()
    print 'Refining grid.. '
    learner.refineGrid()
    learner.learnData()
    print 'Grid size AFTER refinement: %i' % learner.grid.getSize()
    print 'Test accuracy:' + str(learner.testAccuracy)
    print 'L2Norm: %f' % learner.getL2NormError()
    print 'MaxError: %f' % learner.getMaxError()
    print 'MinError: %f' % learner.getMinError()
    print 'Error: %f' % learner.evalError(learner.dataContainer,learner.alpha)
    print 'RMSerror: %f' % learner.error.RMSNorm()
    print '--------------------------------------------'

Test accuracy:[0.017226603272158032]
Level: 1
L2Norm: 1.607480
MaxError: 0.588813
MinError: 0.001064
Error: 0.020180
RMSerror: 0.071566
Grid size BEFORE refinement: 1
Refining grid.. 
Grid size AFTER refinement: 19
Test accuracy:[0.017226603272158032]
L2Norm: 1.604254
MaxError: 0.493885
MinError: 0.000088
Error: 0.004297
RMSerror: 0.017505
--------------------------------------------
Test accuracy:[0.0034818388711247545]
Level: 2
L2Norm: 0.722687
MaxError: 0.306497
MinError: 0.000241
Error: 0.004297
RMSerror: 0.017505
Grid size BEFORE refinement: 19
Refining grid.. 
Grid size AFTER refinement: 199
Test accuracy:[0.0034818388711247545]
L2Norm: 0.859448
MaxError: 0.257240
MinError: 0.000035
Error: 0.001233
RMSerror: 0.004375
--------------------------------------------
